In [7]:
from sqlalchemy.orm import registry
from sqlalchemy import Column, Integer, String, Boolean

In [8]:
mapper_registry = registry()

In [79]:
@mapper_registry.mapped
class Partner:
    __tablename__ = 'partner'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    is_active = Column(Boolean, nullable=False)
    
    def __repr__(self):
        return f'<Partner> {self.id} - {self.name}'

/var/folders/fn/z18mbfyn1qj6tfk92mpyj65h0000gn/T/ipykernel_52923/1829293910.py:2: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Partner, and will be replaced in the string-lookup table.
  class Partner:


In [80]:
# the User class now has a Table object associated with it
Partner.__table__

Table('partner', MetaData(), Column('id', Integer(), table=<partner>, primary_key=True, nullable=False), Column('name', String(), table=<partner>, nullable=False), Column('is_active', Boolean(), table=<partner>, nullable=False), schema=None)

In [81]:
# The Mapper object mediates the relationship between User              
# and the "user" Table object.  This mapper object is generally behind  
# the scenes.
Partner.__mapper__

<Mapper at 0x7fa4a06b8d90; Partner>

In [82]:
boston_wine = Partner(name="Boston Wine Emporium", is_active=True)

In [83]:
boston_wine

<Partner> None - Boston Wine Emporium

In [84]:
boston_wine.is_active

True

In [85]:
boston_wine.name

'Boston Wine Emporium'

In [86]:
# Attributes which we didn't set, such as the "id", are displayed as  |
# None when we access them
repr(boston_wine.id)

'None'

In [87]:
# Using our registry, we can create a database schema for this class using
# a MetaData object that is part of the registry.
from sqlalchemy import create_engine

In [88]:
engine = create_engine("sqlite://")

In [89]:
with engine.begin() as connection:
    mapper_registry.metadata.create_all(connection)

In [90]:
# To persist and load User objects from the database, we           
#  use a Session object, illustrated here from a factory called     
#  sessionmaker.  The Session object makes use of a connection      
#  factory (i.e. an Engine) and will handle the job of connecting,  
#  committing, and releasing connections to this engine.

In [91]:
from sqlalchemy.orm import sessionmaker

In [92]:
Session = sessionmaker(bind=engine, future=True)

In [93]:
session = Session()

In [94]:
# new objects are placed into Session using add()

In [95]:
session.add(boston_wine)

In [96]:
# This did not yet modify the database, however the object is now known as      
# **pending**.  We can see the "pending" objects by looking at the session.new  
# attribute.
session.new

IdentitySet([<Partner> None - Boston Wine Emporium])

In [97]:
boston_wine

<Partner> None - Boston Wine Emporium

In [98]:
from sqlalchemy import select
select_statement = select(Partner).filter_by(name='Boston Wine Emporium')
result = session.execute(select_statement)

In [99]:
result

In [100]:
# We can get the data back from the result, in this case using the 
# .scalar() method which will return the first column of the first row.
also_winery = result.scalar()

In [101]:
also_winery

<Partner> 1 - Boston Wine Emporium

In [103]:
boston_wine.id

1

In [104]:
boston_wine is also_winery

True

In [105]:
session.identity_map.items()

[((__main__.Partner, (1,), None), <Partner> 1 - Boston Wine Emporium)]

In [106]:
session.add_all(
    [
        Partner(name='Arsenal Wine & Spirits', is_active=True),
        Partner(name='Sherry\'s Wine & Spirits', is_active=False)
    ])

In [107]:
boston_wine.is_active = False

In [108]:
# the Session can tell us which objects are dirty
session.dirty

IdentitySet([<Partner> 1 - Boston Wine Emporium])

In [109]:
# the Session can tell us which objects are pending
session.new

IdentitySet([<Partner> None - Arsenal Wine & Spirits, <Partner> None - Sherry's Wine & Spirits])

In [111]:
# The whole transaction is committed.  Commit always triggers
# a final flush of remaining changes.
session.commit()

In [112]:
# After a commit, theres no transaction.  The Session                
# *invalidates* all data, so that accessing them will automatically  
# start a *new* transaction and re-load from the database.  This is  
# our first example of the ORM *lazy loading* pattern.

boston_wine.is_active

False

In [114]:
# *** rolling back changes ***

# You try: 
    # Make another "dirty" change, and another "pending" change,
    # that we might change our minds about.

In [115]:
boston_wine.name = "Fake name for this Boston wine place"
session.add(Partner(name="I dont intend to keep this", is_active=False))

In [116]:
session.dirty

IdentitySet([<Partner> 1 - Fake name for this Boston wine place])

In [117]:
session.new

IdentitySet([<Partner> None - I dont intend to keep this])

In [125]:
result = session.execute(
    select(Partner).where(Partner.name.in_(['Fake name for this Boston wine place', 
                                  'I dont intend to keep this']))
    )


In [126]:
result.all()

[]

In [124]:
session.rollback()

In [127]:
boston_wine.name

'Boston Wine Emporium'

In [128]:
result = session.execute(
    select(Partner).where(Partner.name.in_(['Boston Wine Emporium', 
                                  'I dont intend to keep this']))
    )

In [129]:
result.all()

[(<Partner> 1 - Boston Wine Emporium,)]

In [ ]:
# Notes to myself

# Create engine first with the in-memory db
# Then create a session object
    # How do you see 'dirty' objects? 
    # How do you see 'pending' objects?
    # Check session by running: session.dirty and session.new
    # There are 5 different object states? Need to rewatch the Sqlalchemy 2021 video again
# Before you commit, you can session.rollback()